<a href="https://colab.research.google.com/github/Bahodir011/Bahodir011/blob/main/ml1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import pandas as pd
import numpy as np
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt
import re
%matplotlib inline


df = pd.read_csv('https://raw.githubusercontent.com/anvarnarz/praktikum_datasets/main/housing_data_08-02-2021.csv')


from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(df, test_size=0.2, random_state=42)

housing = train_set.drop("price", axis=1)
housing_labels = train_set["price"].copy()

housing_num = housing.drop(["location", 'district'], axis=1)

housing.drop(housing[housing['size'] == 'Площадьземли:1сот'].index, axis = 0, inplace = True)
housing_num.drop(housing_num[housing_num['size'] == 'Площадьземли:1сот'].index, axis = 0, inplace = True)
housing['size'] = housing['size'].astype(float)
#housing_labels.drop(housing_labels[housing_labels['price'] == 'Договорная'].index, axis = 0, inplace = True)
#housing_labels['price'] = housing_labels['price'].astype(float)

In [25]:
from sklearn.base import BaseEstimator, TransformerMixin

# bizga kerak ustunlar indekslari
rooms_ix, size_ix = 2,3

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_size_per_room = True):
        self.add_size_per_room = add_size_per_room
    def fit(self, X, y=None):
        return self # bizni funksiyamiz faqat transformer. estimator emas
    def transform(self, X):
        size_rooms = X[:, size_ix] / X[:, rooms_ix]

        if self.add_size_per_room: # add_bedrooms_per_room ustuni ixtiyoriy bo'ladi
            size_rooms = X[:, size_ix] / X[:, rooms_ix]
            return np.c_[X, size_rooms ]
        else:
            return np.c_[X, size_rooms]

In [9]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [10]:
num_pipeline = Pipeline([
          ('imputer', SimpleImputer(strategy='median')),
          ('attribs_adder', CombinedAttributesAdder(add_size_per_room = True)),
          ('std_scaler', StandardScaler())
])

In [27]:
num_pipeline.fit_transform(housing_num)

array([[-0.57728139, -0.03810278, -0.31269743, -0.78951747, -0.53193698],
       [ 1.25158231,  0.06792808, -0.75863949, -0.40334253,  0.19982708],
       [-1.49171323, -0.04831316, -0.75863949, -0.40334253,  0.19982708],
       ...,
       [-0.57728139, -0.04045902, -1.20458155, -1.17569241,  0.51344026],
       [-0.57728139, -0.03181947, -0.31269743, -0.78951747, -0.53193698],
       [ 0.33715046, -0.00668623, -0.75863949,  1.14135724,  1.45427977]])

In [23]:
from sklearn.compose import ColumnTransformer

num_attribs = list(housing_num)
cat_attribs = ['district', 'location']

full_pipeline = ColumnTransformer([
    ('num', num_pipeline, num_attribs),
    ('cat', OneHotEncoder(), cat_attribs)
])

In [26]:
housing_pr = full_pipeline.fit_transform(housing)

In [28]:
housing_pr[0:5, :]

<5x1416 sparse matrix of type '<class 'numpy.float64'>'
	with 35 stored elements in Compressed Sparse Row format>